In [14]:
import pandas as pd
import numpy as np
from mysql.connector import connect
import json
from os import path
from utils import get_db_creds
import names
import sqlalchemy
from concurrent import futures

In [2]:
db_creds = get_db_creds()
connection = connect(**db_creds)

In [3]:
!ls food_data/

ingr_map.pkl		interactions_validation.csv  RAW_interactions.csv
interactions_test.csv	PP_recipes.csv		     RAW_recipes.csv
interactions_train.csv	PP_users.csv


In [4]:
recipe_df = pd.read_csv("raw_178k_recipes_with_scraped_measurements.csv")

In [5]:
user_df = pd.read_csv("food_data/PP_users.csv")
raw_interactions_df = pd.read_csv("food_data/RAW_interactions.csv")

In [6]:
user_df.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376


In [7]:
recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,food_recipe_url,quantities,measurement_units
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,https://www.food.com/recipe/arriba---baked-win...,"['1', '1 -2', '1 -2', '', '', '', '']","['lb', 'teaspoon', 'teaspoon', '', '', '', '']"
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,https://www.food.com/recipe/a-bit-different--b...,"['1', '1', '3', '1⁄2', '', '2']",['(10 ounce) can prepared pizza crust (or use ...
2,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,https://www.food.com/recipe/alouette--potatoes...,"['2', '12', '2', '2', '1', '5', '2', '1⁄8', '1...","['(4 ounce) packages', 'large', 'shallots', 't..."
3,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['combine ingredients in blender', 'cover and ...",NaN,"['milk', 'vanilla ice cream', 'frozen apple ju...",4,https://www.food.com/recipe/apple-a-day--milk-...,"['1', '1', '2', '1⁄2']","['cup', 'scoop', 'tablespoons', 'apple']"
4,aww marinated olives,25274,15,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,['toast the fennel seeds and lightly crush the...,my italian mil was thoroughly impressed by my ...,"['fennel seeds', 'green olives', 'ripe olives'...",9,https://www.food.com/recipe/aww--marinated-oli...,"['1 1⁄2', '1', '1', '1', '1', '1', '1', '1', '4']","['teaspoons', 'cup', 'cup', 'clove', 'teaspoon..."


In [8]:
raw_interactions_df = raw_interactions_df[raw_interactions_df["recipe_id"].isin(recipe_df["id"])]
raw_interactions_df = raw_interactions_df.sample(random_state=0, n=200000)

In [9]:
raw_interactions_df.head()

,user_id,recipe_id,date,rating,review
1055849,83491,52469,2003-04-24,5,It's GENIUS! I love this recipe- thank you so ...
784868,126440,241877,2009-02-20,4,Good side dish. I only made half a recipe and...
155101,182304,373493,2012-07-07,5,Delicious! I did eyeball the honeydew (used a ...
776190,113509,307866,2011-09-06,5,"Wow, this was such a great recipe. I wasn't su..."
539341,352597,205407,2007-01-16,5,It was both simple and delicious. Although I a...


In [10]:
codes, uniques = pd.factorize(recipe_df["contributor_id"])
codes += 1
recipe_df["contributor_id"] = codes

In [11]:
max(codes)

22218

In [12]:
raw_interactions_df["user_id"] = np.random.choice(recipe_df["contributor_id"], 200000)

In [18]:
def generate_name(i):
    return names.get_full_name()
MAX_PROCESSES = 30
with futures.ProcessPoolExecutor(max_workers=MAX_PROCESSES) as executor:
    user_names = list(executor.map(names.get_full_name, [i for i in range(max(codes))]))

In [21]:
db_creds.keys()

dict_keys(['host', 'user', 'password', 'database'])

In [22]:
users_df = pd.DataFrame({"user_id": sorted(np.unique(codes)), "username": user_names})
users_df["hashed_password"] = "password"

In [23]:
conn = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'
            .format(db_creds["user"], db_creds["password"], db_creds["host"], db_creds["database"])).connect()

In [24]:
users_df.to_sql(name="SampleUser", con=conn, if_exists="append", index=False,)

In [30]:
users_df.to_csv("users.csv", index=False)

In [31]:
raw_interactions_df.to_csv("interactions_with_updated_users.csv", index=False)

In [32]:
recipe_df.to_csv("recipe_df_178k_with_updated_users.csv", index=False)